In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections

In [2]:
# Copyright (c) 2017-2019 The University of Manchester
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

from enum import Enum
import struct

from spinn_front_end_common.abstract_models import AbstractProvidesNKeysForPartition
from spinn_utilities.overrides import overrides
from pacman.executor.injection_decorator import inject_items
from pacman.model.graphs.machine import MachineVertex
from pacman.model.resources import ResourceContainer, VariableSDRAM
from pacman.utilities.utility_calls import is_single
from spinn_front_end_common.utilities.constants import (
    SYSTEM_BYTES_REQUIREMENT, BYTES_PER_WORD)
from spinn_front_end_common.utilities.exceptions import ConfigurationException
from spinn_front_end_common.utilities.helpful_functions import (
    locate_memory_region_for_placement)
from spinn_front_end_common.abstract_models.impl import (
    MachineDataSpecableVertex)
from spinn_front_end_common.interface.buffer_management.buffer_models import (
    AbstractReceiveBuffersToHost)
from spinn_front_end_common.interface.buffer_management import (
    recording_utilities)
from spinnaker_graph_front_end.utilities import SimulatorVertex
from spinnaker_graph_front_end.utilities.data_utils import (
    generate_system_data_region)

from data_specification.enums.data_type import DataType


class LatticeBasicCell(
    SimulatorVertex, MachineDataSpecableVertex,
    AbstractReceiveBuffersToHost,
    AbstractProvidesNKeysForPartition
):
    """ Cell which represents a cell within the 2d fabric
    """

    PARTITION_ID = "STATE"

    TRANSMISSION_DATA_SIZE = 2 * BYTES_PER_WORD  # has key and key
    STATE_DATA_SIZE = BYTES_PER_WORD  # 1 or 2 based off dead or alive
    # alive states, dead states
    NEIGHBOUR_INITIAL_STATES_SIZE = 2 * BYTES_PER_WORD
    RECORDING_ELEMENT_SIZE = STATE_DATA_SIZE  # A recording of the state

    POSITION_DATA_SIZE = 2 * BYTES_PER_WORD

    NEIGHBOUR_KEYS_SIZE = 8 * BYTES_PER_WORD

    VELOCITY_SIZE = 2 * BYTES_PER_WORD  # u_x and u_y

    # Regions for populations
    DATA_REGIONS = Enum(
        value="DATA_REGIONS",
        names=[('SYSTEM', 0),
               ('TRANSMISSIONS', 1),
               ('POSITION', 2),
               ('NEIGHBOUR_KEYS', 3),
               ('VELOCITY', 4),
               ('RESULTS', 5)])

    def __init__(self, label, x_position, y_position, n_key, w_key, s_key, e_key, nw_key, sw_key, se_key, ne_key, u_x,
                 u_y):
        super(LatticeBasicCell, self).__init__(label, "lattice_cell.aplx")
        AbstractProvidesNKeysForPartition.__init__(self)
        # app specific data items
        self._x_position = x_position
        self._y_position = y_position
        self._n_key = n_key
        self._s_key = s_key
        self._w_key = w_key
        self._e_key = e_key
        self._nw_key = nw_key
        self._ne_key = ne_key
        self._sw_key = sw_key
        self._se_key = se_key
        self.u_x = u_x
        self.u_y = u_y


    @inject_items({"data_n_time_steps": "DataNTimeSteps"})
    @overrides(
        MachineDataSpecableVertex.generate_machine_data_specification,
        additional_arguments={"data_n_time_steps"})
    def generate_machine_data_specification(
            self, spec, placement, machine_graph, routing_info, iptags,
            reverse_iptags, machine_time_step, time_scale_factor,
            data_n_time_steps):
        # Generate the system data region for simulation .c requirements
        generate_system_data_region(spec, self.DATA_REGIONS.SYSTEM.value,
                                    self, machine_time_step, time_scale_factor)

        # reserve memory regions
        spec.reserve_memory_region(
            region=self.DATA_REGIONS.TRANSMISSIONS.value,
            size=self.TRANSMISSION_DATA_SIZE, label="inputs")

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.POSITION.value,
            size=self.POSITION_DATA_SIZE, label="position"
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.NEIGHBOUR_KEYS.value,
            size=self.NEIGHBOUR_KEYS_SIZE
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.VELOCITY.value,
            size=self.VELOCITY_SIZE
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.RESULTS.value,
            size=recording_utilities.get_recording_header_size(1))

        # get recorded buffered regions sorted
        spec.switch_write_focus(self.DATA_REGIONS.RESULTS.value)
        spec.write_array(recording_utilities.get_recording_header_array(
            [self.RECORDING_ELEMENT_SIZE * data_n_time_steps]))

        # check got right number of keys and edges going into me
        partitions = \
            machine_graph.get_outgoing_edge_partitions_starting_at_vertex(self)
        if not is_single(partitions):
            raise ConfigurationException(
                "Can only handle one type of partition.")

        # check for duplicates
        edges = list(machine_graph.get_edges_ending_at_vertex(self))
        if len(edges) != 8:
            raise ConfigurationException(
                "I've not got the right number of connections. I have {} "
                "instead of 8".format(
                    len(machine_graph.get_edges_ending_at_vertex(self))))

        for edge in edges:
            if edge.pre_vertex == self:
                raise ConfigurationException(
                    "I'm connected to myself, this is deemed an error"
                    " please fix.")

        # write key needed to transmit with
        key = routing_info.get_first_key_from_pre_vertex(
            self, self.PARTITION_ID)

        spec.switch_write_focus(
            region=self.DATA_REGIONS.TRANSMISSIONS.value)
        spec.write_value(0 if key is None else 1)
        spec.write_value(0 if key is None else key)

        # write POSITION data
        spec.switch_write_focus(
            region=self.DATA_REGIONS.POSITION.value
        )
        spec.write_value(int(self._x_position))
        spec.write_value(int(self._y_position))
        # write neighbour keys
        # diraction = ["me", "n", "w", "s", "e", "nw", "sw", "se", "ne"]
        spec.switch_write_focus(
            region=self.DATA_REGIONS.NEIGHBOUR_KEYS.value
        )
        spec.write_value(self._n_key)
        spec.write_value(self._w_key)
        spec.write_value(self._s_key)
        spec.write_value(self._e_key)
        spec.write_value(self._nw_key)
        spec.write_value(self._sw_key)
        spec.write_value(self._se_key)
        spec.write_value(self._ne_key)

        spec.switch_write_focus(region=self.DATA_REGIONS.VELOCITY.value)
        spec.write_value(self.u_x, data_type=DataType.FLOAT_32)
        spec.write_value(self.u_y, data_type=DataType.FLOAT_32)

        # End-of-Spec:
        spec.end_specification()

    def get_data(self, buffer_manager, placement):
        # for buffering output info is taken form the buffer manager
        # get raw data, convert to list of booleans
        raw_data, data_missing = buffer_manager.get_data_by_placement(
            placement, 0)

        # do check for missing data
        if data_missing:
            print("missing_data from ({}, {}, {}); ".format(
                placement.x, placement.y, placement.p))

        # return the data, converted to list of booleans
        return [
            element
            for element in struct.unpack(
                "<{}f".format(len(raw_data) // BYTES_PER_WORD), raw_data)]

    @property
    @overrides(MachineVertex.resources_required)
    def resources_required(self):
        fixed_sdram = (SYSTEM_BYTES_REQUIREMENT + self.TRANSMISSION_DATA_SIZE +
                       self.POSITION_DATA_SIZE +
                       self.NEIGHBOUR_KEYS_SIZE +
                       self.VELOCITY_SIZE +
                       recording_utilities.get_recording_header_size(1) +
                       recording_utilities.get_recording_data_constant_size(1))
        per_timestep_sdram = self.RECORDING_ELEMENT_SIZE
        return ResourceContainer(
            sdram=VariableSDRAM(fixed_sdram, per_timestep_sdram))

    @property
    def x_position(self):
        return self._x_position

    @overrides(AbstractProvidesNKeysForPartition.get_n_keys_for_partition)
    def get_n_keys_for_partition(self, partition, graph_mapper):
        return 8  # one for  n w s e nw ne sw se

    @property
    def y_position(self):
        return self._y_position

    def __repr__(self):
        return self.label

    @overrides(AbstractReceiveBuffersToHost.get_recorded_region_ids)
    def get_recorded_region_ids(self):
        return [0]

    @overrides(AbstractReceiveBuffersToHost.get_recording_region_base_address)
    def get_recording_region_base_address(self, txrx, placement):
        return locate_memory_region_for_placement(
            placement, self.DATA_REGIONS.RESULTS.value, txrx)


In [6]:
# Copyright (c) 2017-2019 The University of Manchester
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
import math
import os
from pacman.model.graphs.machine import MachineEdge
import spinnaker_graph_front_end as front_end

runtime = 500
# machine_time_step = 100
MAX_X_SIZE_OF_FABRIC = 10
MAX_Y_SIZE_OF_FABRIC = 10
n_chips = (MAX_X_SIZE_OF_FABRIC * MAX_Y_SIZE_OF_FABRIC) // 15

ex = [0, 1, 0, -1, 0, 1, -1, -1, 1]
ey = [0, 0, 1, 0, -1, 1, 1, -1, -1]


def generatePositionMap(x_dim, y_dim):
    positionMap = {}
    #   diraction = ["me", "n", "w", "s", "e", "nw", "sw", "se", "ne"]
    for i in range(9):
        x_temp = int(x_dim - ex[i]) % MAX_X_SIZE_OF_FABRIC
        y_temp = int(y_dim - ey[i]) % MAX_Y_SIZE_OF_FABRIC
        positionMap[i] = (x_temp, y_temp)
    return positionMap


def generateKeys(position):
    return position[0] * MAX_X_SIZE_OF_FABRIC + position[1]


def initVelocity(x_pos, y_pos):
    U_0 = 0.01
    K = 30.0
    delta = 0.05
    x_temp = 1.0 * (x_pos) / MAX_X_SIZE_OF_FABRIC
    y_temp = 1.0 * (y_pos) / MAX_Y_SIZE_OF_FABRIC
    if y_temp <= 0.5:
        u_x = U_0 * math.tanh(K * (y_temp - 0.25))
    else:
        u_x = U_0 * math.tanh(K * (0.75 - y_temp))
#     print("the u_x of ({}, {}) is {}".format(x_pos, y_pos, u_x))
    u_y = U_0 * delta * math.sin(2 * math.pi * (x_temp + 0.25))
#     print("the u_y of ({}, {}) is {}".format(x_pos, y_pos, u_y))    
    return u_x, u_y

# set up the front end and ask for the detected machines dimensions
front_end.setup(
    n_chips_required=n_chips, model_binary_folder=os.path.dirname(os.path.abspath("__file__")), machine_time_step=10000)

# figure out if machine can handle simulation
cores = front_end.get_number_of_available_cores_on_machine()
# print(cores)
if cores <= (MAX_X_SIZE_OF_FABRIC * MAX_Y_SIZE_OF_FABRIC):
    raise KeyError("Don't have enough cores to run simulation")

# contain the vertices for the connection aspect
vertices = [
    [None for _ in range(MAX_X_SIZE_OF_FABRIC)]
    for _ in range(MAX_Y_SIZE_OF_FABRIC)]

# build vertices
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
        u_x, u_y = initVelocity(x, y)
        keymap = generatePositionMap(x, y)
        vert = LatticeBasicCell(
            "cell{}".format((x * MAX_X_SIZE_OF_FABRIC) + y),
            x, y, generateKeys(keymap[1]), generateKeys(keymap[2]), generateKeys(keymap[3]), generateKeys(keymap[4]),
            generateKeys(keymap[5]), generateKeys(keymap[6]), generateKeys(keymap[7]),
            generateKeys(keymap[8]), u_x, u_y)
        vertices[x][y] = vert
        front_end.add_machine_vertex_instance(vert)

# verify the initial state
output = ""
for y in range(MAX_X_SIZE_OF_FABRIC - 1, 0, -1):
    for x in range(0, MAX_Y_SIZE_OF_FABRIC):
        output += "({0}, {1}) ".format(vertices[x][y].x_position, vertices[x][y].y_position)
    output += "\n"
print(output)
print("\n\n")

# build edges
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
        keymap = generatePositionMap(x, y)
        #   diraction = ["me", "n", "w", "s", "e", "nw", "sw", "se", "ne"]
        positions = [
            (x, (y + 1) % MAX_Y_SIZE_OF_FABRIC, "E"),
            ((x + 1) % MAX_X_SIZE_OF_FABRIC,
             (y + 1) % MAX_Y_SIZE_OF_FABRIC, "SE"),
            ((x + 1) % MAX_X_SIZE_OF_FABRIC, y, "S"),
            ((x + 1) % MAX_X_SIZE_OF_FABRIC,
             (y - 1) % MAX_Y_SIZE_OF_FABRIC, "SW"),
            (x, (y - 1) % MAX_Y_SIZE_OF_FABRIC, "W"),
            ((x - 1) % MAX_X_SIZE_OF_FABRIC,
             (y - 1) % MAX_Y_SIZE_OF_FABRIC, "NW"),
            ((x - 1) % MAX_X_SIZE_OF_FABRIC, y, "N"),
            ((x - 1) % MAX_X_SIZE_OF_FABRIC,
             (y + 1) % MAX_Y_SIZE_OF_FABRIC, "NE")]

        for (dest_x, dest_y, compass) in positions:
            front_end.add_machine_edge_instance(
                MachineEdge(
                    vertices[x][y], vertices[dest_x][dest_y],
                    label=compass), LatticeBasicCell.PARTITION_ID)

# run the simulation
front_end.run(runtime)

# get recorded data
recorded_data = dict()

# if not front_end.use_virtual_machine():
buffer_manager = front_end.buffer_manager()

# get the data per vertex
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
        recorded_data[x, y] = vertices[x][y].get_data(
            front_end.buffer_manager(),
            front_end.placements().get_placement_of_vertex(
                vertices[x][y]))

# visualise it in text form (bad but no vis this time)
for time in range(0, runtime, 2):
    print("at time {}".format(time))
    output = ""
    for x in range(0, MAX_Y_SIZE_OF_FABRIC):
        for y in range(0, MAX_Y_SIZE_OF_FABRIC):
            output += "{}{}".format(recorded_data[x, y][time], recorded_data[x, y][time+1])
        output += "\n"
    print(output)
    print("\n\n")

# clear the machine
front_end.stop()


2020-07-07 17:07:56 INFO: SpiNNaker graph front end (c) 2019, University of Manchester
2020-07-07 17:07:56 INFO: Release version 1!5.1.0(Liveware Problem) - November 2019. Installed in folder /home/spinnaker/sPyNNaker/lib/python3.6/site-packages
2020-07-07 17:07:56 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinnaker_graph_front_end/spiNNakerGraphFrontEnd.cfg, /home/spinnaker/.spiNNakerGraphFrontEnd.cfg
2020-07-07 17:07:56 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker
2020-07-07 17:07:56 WARNING: /home/spinnaker/reports has 52 old reports that have not been closed
2020-07-07 17:07:56 WARNING: /home/spinnaker/application_generated_data_files has 36 old reports that have not been closed
2020-07-07 17:07:56 INFO: Setting time scale factor 

['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinnaker_graph_front_end/spiNNakerGraphFrontEnd.cfg', '/home/spinnaker/.spiNNakerGraphFrontEnd.cfg']


2020-07-07 17:08:05 INFO: Time 0:00:08.940781 taken by SpallocAllocator
2020-07-07 17:08:05 INFO: Creating transceiver for 10.11.196.1
2020-07-07 17:08:05 INFO: Working out if machine is booted
2020-07-07 17:08:09 INFO: Attempting to boot machine
2020-07-07 17:08:15 INFO: Found board with version [Version: SC&MP 3.2.5 at SpiNNaker:0:0:0 (built Thu Aug  1 08:15:06 2019)]
2020-07-07 17:08:15 INFO: Machine communication successful
2020-07-07 17:08:15 INFO: Detected a machine on IP address 10.11.196.1 which has 855 cores and 120.0 links
2020-07-07 17:08:15 INFO: Time 0:00:09.635746 taken by MachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-07-07 17:08:15 INFO: Time 0:00:00.022717 taken by PreAllocateResourcesForExtraMonitorSupport
2020-07-07 17:08:15 INFO: Starting execution process
2020-07-07 17:08:15 INFO: Time 0:00:00.001621 taken by NetworkSpecificationReport
Allocating virtual identifiers
|0

(0, 9) (1, 9) (2, 9) (3, 9) (4, 9) (5, 9) (6, 9) (7, 9) (8, 9) (9, 9) 
(0, 8) (1, 8) (2, 8) (3, 8) (4, 8) (5, 8) (6, 8) (7, 8) (8, 8) (9, 8) 
(0, 7) (1, 7) (2, 7) (3, 7) (4, 7) (5, 7) (6, 7) (7, 7) (8, 7) (9, 7) 
(0, 6) (1, 6) (2, 6) (3, 6) (4, 6) (5, 6) (6, 6) (7, 6) (8, 6) (9, 6) 
(0, 5) (1, 5) (2, 5) (3, 5) (4, 5) (5, 5) (6, 5) (7, 5) (8, 5) (9, 5) 
(0, 4) (1, 4) (2, 4) (3, 4) (4, 4) (5, 4) (6, 4) (7, 4) (8, 4) (9, 4) 
(0, 3) (1, 3) (2, 3) (3, 3) (4, 3) (5, 3) (6, 3) (7, 3) (8, 3) (9, 3) 
(0, 2) (1, 2) (2, 2) (3, 2) (4, 2) (5, 2) (6, 2) (7, 2) (8, 2) (9, 2) 
(0, 1) (1, 1) (2, 1) (3, 1) (4, 1) (5, 1) (6, 1) (7, 1) (8, 1) (9, 1) 






2020-07-07 17:08:15 INFO: Time 0:00:00.052201 taken by InsertEdgesToExtraMonitorFunctionality
Generating routing tables for data in system processes
|0%                          50%                         100%|
2020-07-07 17:08:15 INFO: Time 0:00:00.009551 taken by DataInMulticastRoutingGenerator
Generating fixed router routes
|0%                          50%                         100%|
2020-07-07 17:08:15 INFO: Time 0:00:00.008069 taken by FixedRouteRouter
Routing
|0%                          50%                         100%|

2020-07-07 17:08:15 INFO: Time 0:00:00.063987 taken by NerRoute
Discovering tags
|0%                          50%                         100%|
Allocating tags
|0%                          50%                         100%|
2020-07-07 17:08:15 INFO: Time 0:00:00.059822 taken by BasicTagAllocator
Reporting Tags
|0%                          50%                         100%|
2020-07-07 17:08:15 INFO: Time 0:00:00.007771 taken by TagReport
Getting number of keys r

...

2020-07-07 17:08:44 ERROR: PSR=0x0000001F SR=0x0040FAA8 LR=0x00005493
2020-07-07 17:08:44 ERROR: 1, 0, 3: RUN_TIME_EXCEPTION (IOBUF) lattice_cell
2020-07-07 17:08:44 ERROR: r0=0x0000000E r1=0x80000000 r2=0x67FFF770 r3=0x00000001
2020-07-07 17:08:44 ERROR: r4=0xE5007F00 r5=0x004002B4 r6=0x67FCB640 r7=0x00000020
2020-07-07 17:08:44 ERROR: PSR=0x0000001F SR=0x0040FAA8 LR=0x00005493
2020-07-07 17:08:44 ERROR: 1, 0, 4: RUN_TIME_EXCEPTION (IOBUF) lattice_cell
2020-07-07 17:08:44 ERROR: r0=0x0000000E r1=0x80000000 r2=0x67FFF770 r3=0x00000001
2020-07-07 17:08:44 ERROR: r4=0xE5007F00 r5=0x004002B4 r6=0x67FCF658 r7=0x0040FAE0
2020-07-07 17:08:44 ERROR: PSR=0x0000001F SR=0x0040FAA8 LR=0x00005493
2020-07-07 17:08:44 ERROR: 1, 0, 5: RUN_TIME_EXCEPTION (IOBUF) lattice_cell
2020-07-07 17:08:44 ERROR: r0=0x0000000E r1=0x80000000 r2=0x67FFF770 r3=0x00000001
2020-07-07 17:08:44 ERROR: r4=0xE5007F00 r5=0x004002B4 r6=0x67FC3610 r7=0x00000020
2020-07-07 17:08:44 ERROR: PSR=0x000000DF SR=0x0040FA90 LR=0x000

2020-07-07 17:08:44 ERROR: 1, 1, 13: RUN_TIME_EXCEPTION (SWERR) lattice_cell
2020-07-07 17:08:44 ERROR: r0=0x0000000D r1=0x00000000 r2=0x00004000 r3=0x00000550
2020-07-07 17:08:44 ERROR: r4=0x00400934 r5=0x0000000E r6=0x0000008E r7=0x00400000
2020-07-07 17:08:44 ERROR: PSR=0x600000DF SR=0x0040FB38 LR=0x00000558
2020-07-07 17:08:44 ERROR: 0, 1, 2: RUN_TIME_EXCEPTION (IOBUF) lattice_cell
2020-07-07 17:08:44 ERROR: r0=0x0000000E r1=0x80000000 r2=0x67FFF770 r3=0x00000001
2020-07-07 17:08:44 ERROR: r4=0xE5007F00 r5=0x004002B4 r6=0x67FCF658 r7=0x00000020
2020-07-07 17:08:44 ERROR: PSR=0x000000DF SR=0x0040FAA8 LR=0x00005493
2020-07-07 17:08:44 ERROR: 1, 1, 14: RUN_TIME_EXCEPTION (IOBUF) lattice_cell
2020-07-07 17:08:44 ERROR: r0=0x0000000E r1=0x80000000 r2=0x67FFF770 r3=0x00000001
2020-07-07 17:08:44 ERROR: r4=0xE5007F00 r5=0x004002B4 r6=0x67FEF718 r7=0x0040FAE0
2020-07-07 17:08:44 ERROR: PSR=0x0000001F SR=0x0040FAA8 LR=0x00005493
2020-07-07 17:08:44 ERROR: 0, 1, 3: RUN_TIME_EXCEPTION (IOBUF)

2020-07-07 17:08:44 ERROR: r0=0x0000000E r1=0x80000000 r2=0x67FFF770 r3=0x00000001
2020-07-07 17:08:44 ERROR: r4=0xE5007F00 r5=0x004002B4 r6=0x67FF7748 r7=0x00000020
2020-07-07 17:08:44 ERROR: PSR=0x0000001F SR=0x0040FAA8 LR=0x00005493
2020-07-07 17:08:44 ERROR: 2, 0, 9: RUN_TIME_EXCEPTION (IOBUF) lattice_cell
2020-07-07 17:08:44 ERROR: r0=0x0000000E r1=0x80000000 r2=0x67FFF770 r3=0x00000001
2020-07-07 17:08:44 ERROR: r4=0xE5007F00 r5=0x004002B4 r6=0x67FEB700 r7=0x00000020
2020-07-07 17:08:44 ERROR: PSR=0x0000001F SR=0x0040FAA8 LR=0x00005493
2020-07-07 17:08:44 ERROR: 2, 0, 10: RUN_TIME_EXCEPTION (IOBUF) lattice_cell
2020-07-07 17:08:44 ERROR: r0=0x0000000E r1=0x80000000 r2=0x67FFF770 r3=0x00000001
2020-07-07 17:08:44 ERROR: r4=0xE5007F00 r5=0x004002B4 r6=0x67FF3730 r7=0x0040FAE0
2020-07-07 17:08:44 ERROR: PSR=0x0000001F SR=0x0040FAA8 LR=0x00005493
2020-07-07 17:08:44 ERROR: 2, 0, 11: RUN_TIME_EXCEPTION (IOBUF) lattice_cell
2020-07-07 17:08:44 ERROR: r0=0x0000000E r1=0x80000000 r2=0x67

2020-07-07 17:08:44 ERROR: r4=0xE5007F00 r5=0x004002B4 r6=0x67FCF658 r7=0x00000020
2020-07-07 17:08:44 ERROR: PSR=0x0000001F SR=0x0040FAA8 LR=0x00005493
2020-07-07 17:08:44 ERROR: 2, 2, 4: RUN_TIME_EXCEPTION (SWERR) lattice_cell
2020-07-07 17:08:44 ERROR: r0=0x0000000D r1=0x00000000 r2=0x00004000 r3=0x00000550
2020-07-07 17:08:44 ERROR: r4=0x00400934 r5=0x0000000E r6=0x0000008E r7=0x00400000
2020-07-07 17:08:44 ERROR: PSR=0x600000DF SR=0x0040FB38 LR=0x00000558
2020-07-07 17:08:44 ERROR: 2, 1, 17: RUN_TIME_EXCEPTION (IOBUF) lattice_cell
2020-07-07 17:08:44 ERROR: r0=0x0000000E r1=0x80000000 r2=0x67FFF770 r3=0x00000001
2020-07-07 17:08:44 ERROR: r4=0xE5007F00 r5=0x004002B4 r6=0x67FCB640 r7=0x00000020
2020-07-07 17:08:44 ERROR: PSR=0x0000001F SR=0x0040FAA8 LR=0x00005493
2020-07-07 17:08:44 ERROR: 2, 2, 5: RUN_TIME_EXCEPTION (SWERR) lattice_cell
2020-07-07 17:08:44 ERROR: r0=0x0000000D r1=0x00000000 r2=0x00004000 r3=0x00000550
2020-07-07 17:08:44 ERROR: r4=0x00400934 r5=0x0000000E r6=0x000

SpiNNManCoresNotInStateException: Operation waiting for cores to reach one of [<CPUState.READY: 5>] timed out after 5 seconds

In [ ]:
# visualise it in text form (bad but no vis this time)
u_x_output = collections.defaultdict(dict)
u_y_output = collections.defaultdict(dict)
for i in range(0, MAX_X_SIZE_OF_FABRIC):
    for j in range(0, MAX_Y_SIZE_OF_FABRIC):
        u_x_output[i][j] = recorded_data[i, j][98]
        u_y_output[i][j] = recorded_data[i, j][99]
u_x_output

In [ ]:
u_y_output

In [ ]:
voriticity = collections.defaultdict(dict)

for i in range(0, MAX_X_SIZE_OF_FABRIC):
    for j in range(0, MAX_Y_SIZE_OF_FABRIC): 
        voriticity[i][j] = 0.5 * (u_y_output[(i + 1) % MAX_X_SIZE_OF_FABRIC][j] - u_y_output[(i - 1)%MAX_X_SIZE_OF_FABRIC][j]) + 0.5 * (u_x_output[i][(j + 1)%MAX_Y_SIZE_OF_FABRIC] - u_x_output[i][(j - 1) % MAX_Y_SIZE_OF_FABRIC])

voriticity